<a href="https://colab.research.google.com/github/MuhammadIrzam447/visionCodes/blob/master/mm_imdb(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown

In [ ]:
!gdown

In [ ]:
!gdown

In [ ]:
!gdown https://drive.google.com/file/d/17u9uNTTtdEJI8ZGgHAX599lALj7a4ey_/view?usp=sharing

In [ ]:
!unzip test.zip

In [ ]:
!unzip train.zip

# New Section

Training Dataset

In [ ]:
import os
train_image_file_paths = []
train_genre_labels = []

train_image_folder_add = "/content/train"
train_labels_file = "/content/train_label.txt"

with open(train_labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces
        image_path = os.path.join(train_image_folder_add, filename)
        train_image_file_paths.append(image_path)
        train_genre_labels.append(labels)

In [ ]:
len(train_image_file_paths)

In [ ]:
from collections import defaultdict

label_counts = defaultdict(int)

for labels in train_genre_labels:
    for label in labels:
        label_counts[label] += 1

In [ ]:
label_counts

In [ ]:
# Convert the defaultdict into a list of tuples
label_count_list = [(label, count) for label, count in label_counts.items()]

# Sort the list of tuples based on counts in descending order
sorted_label_count_list = sorted(label_count_list, key=lambda x: x[1], reverse=True)

# Display the sorted list
for label, count in sorted_label_count_list:
    print(f"{label}: {count}")

print("Total Labels: ", len(label_count_list))

In [ ]:
min_label_count = 500
valid_labels = [label for label, count in label_counts.items() if count >= min_label_count]
valid_labels = sorted(list(valid_labels))

In [ ]:
valid_labels, len(valid_labels)

In [ ]:
num_labels = len(valid_labels)
label2id = {label: str(i) for i, label in enumerate(valid_labels)}
id2label = {str(i): label for i, label in enumerate(valid_labels)}

In [ ]:
id2label

In [ ]:
num_labels

In [ ]:
train_multi_hot_labels = []

for labels in train_genre_labels:
    multi_hot = [1 if label in labels else 0 for label in valid_labels]
    train_multi_hot_labels.append(multi_hot)

In [ ]:
train_multi_hot_labels[0]

In [ ]:
train_genre_labels[0]

Test Dataset

In [ ]:
import os
image_file_paths = []
genre_labels = []

image_folder_add = "/content/test"
labels_file = '/content/test_label.txt'

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')  # Split labels by comma and remove leading/trailing spaces
        image_path = os.path.join(image_folder_add, filename)
        image_file_paths.append(image_path)
        genre_labels.append(labels)

In [ ]:
len(image_file_paths)

In [ ]:
# from collections import defaultdict

# label_counts = defaultdict(int)

# for labels in genre_labels:
#     for label in labels:
#         label_counts[label] += 1

In [ ]:
# label_counts

In [ ]:
# label_count_list = [(label, count) for label, count in label_counts.items()]
# sorted_label_count_list = sorted(label_count_list, key=lambda x: x[1], reverse=True)
# for label, count in sorted_label_count_list:
#     print(f"{label}: {count}")
# print("Total Labels: ", len(label_count_list))

In [ ]:
multi_hot_labels = []

for labels in genre_labels:
    multi_hot = [1 if label in labels else 0 for label in valid_labels]
    multi_hot_labels.append(multi_hot)

Creating Datasets

In [ ]:
!pip install datasets
from datasets import Dataset

In [ ]:
train_data = {'image': train_image_file_paths, 'label': train_multi_hot_labels}
ds_train = Dataset.from_dict(train_data)

val_data = {'image': image_file_paths, 'label': multi_hot_labels}
ds_val = Dataset.from_dict(val_data)

In [ ]:
ds_train

In [ ]:
ds_val

In [ ]:
import os

def filter_function(example):
    filename = example["image"]
    return not (filename.endswith("_1.png") or filename.endswith("_2.png"))

In [ ]:
ds_val = ds_val.filter(filter_function)

In [ ]:
ds_val

In [ ]:
ds_train['image'][0], ds_val['image'][0]

In [ ]:
# labels = ds.features["label"]
labels = valid_labels
labels

Creating Model

In [ ]:
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
model_name = "google/vit-base-patch16-224"
image_processor = ViTImageProcessor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name).to(device)

In [ ]:
import PIL.Image as pil

def transform(examples):
  # inputs = image_processor([Image.open(img).convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs = image_processor([pil.open(img).convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]
  return inputs

In [ ]:
train_dataset = ds_train.with_transform(transform)
val_dataset = ds_val.with_transform(transform)

In [ ]:
train_dataset

In [ ]:
val_dataset

In [ ]:
sample = train_dataset[0]
# img = sample["pixel_values"]
# filename = os.path.basename(img.filename)
# print(f"File Name: {filename}")

In [ ]:
sample

In [ ]:
for item in train_dataset:
  print(item['pixel_values'].shape)
  print(item["labels"])
  break

In [ ]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

Attach Final Layer to Model

In [ ]:
# load the ViT model
model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels= len(valid_labels),
    label2id = {label: str(i) for i, label in enumerate(valid_labels)},
    id2label = {str(i): label for i, label in enumerate(valid_labels)},
    problem_type = "multi_label_classification",
    ignore_mismatched_sizes=True,
)

Setting Hyperparameters

In [ ]:
# Training loop
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader

batch_size = 32

In [ ]:
train_dataset_loader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
valid_dataset_loader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
log_dir = "/content/Untitled Folder"
summary_writer = SummaryWriter(log_dir=log_dir)

In [ ]:
num_epochs = 20
model = model.to(device)

In [ ]:
n_train_steps = num_epochs * len(train_dataset_loader)
n_valid_steps = len(valid_dataset_loader)
current_step = 0

In [ ]:
n_train_steps , n_valid_steps

In [ ]:
# logging, eval & save steps
save_steps = 1000

In [ ]:
# def compute_metrics(eval_pred):
#   accuracy_score = accuracy.compute(predictions=eval_pred.predictions, references=eval_pred.label_ids)
#   f1_score = f1.compute(predictions=eval_pred.predictions, references=eval_pred.label_ids, average="macro")
#   return {**accuracy_score, **f1_score}

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

for epoch in range(num_epochs):

    model.train()
    train_loss = 0
    progress_bar = tqdm(range(current_step, n_train_steps), "Training", dynamic_ncols=True, ncols=80)

    for batch in train_dataset_loader:
      if (current_step+1) % save_steps == 0:
        print()
        print(f"Validation at step {current_step}...")
        print()

        model.eval()

        predictions, labels = [], []
        valid_loss = 0

        for batch in valid_dataset_loader:
            pixel_values = batch["pixel_values"].to(device)
            label_ids = batch["labels"].to(device).float()

            outputs = model(pixel_values=pixel_values, labels=label_ids)

            loss = outputs.loss
            valid_loss += loss.item()

            logits = outputs.logits.detach().cpu()

            # predictions.extend(logits.argmax(dim=-1).tolist())
            # labels.extend(label_ids.tolist())
            predictions.extend((logits > 0.5).int().cpu().numpy())
            labels.extend(label_ids.int().cpu().numpy())

        # eval_prediction = EvalPrediction(predictions=predictions, label_ids=labels)
        # metrics = compute_metrics(eval_prediction)
        print()
        print(f"Epoch: {epoch}, Step: {current_step}, Train Loss: {train_loss / save_steps:.4f}, " +
              f"Valid Loss: {valid_loss / n_valid_steps:.4f}") # , Accuracy: {metrics['accuracy']}, " + f"F1 Score: {metrics['f1']}")
        print()

        # summary_writer.add_scalar("valid_loss", valid_loss / n_valid_steps, global_step=current_step)
        # summary_writer.add_scalar("accuracy", metrics["accuracy"], global_step=current_step)
        # summary_writer.add_scalar("f1", metrics["f1"], global_step=current_step)

        model.save_pretrained(f"/content/checkpoint-{current_step}")
        image_processor.save_pretrained(f"/content/checkpoint-{current_step}")

        accuracy = accuracy_score(labels, predictions)
        precision = precision_score(labels, predictions, average='macro')
        recall = recall_score(labels, predictions, average='macro')
        f1 = f1_score(labels, predictions, average='macro')

        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1-Score: {f1}")
        print(classification_report(labels, predictions))

        model.train()
        train_loss, valid_loss = 0, 0

      pixel_values = batch["pixel_values"].to(device)
      labels = batch["labels"].to(device).float()

      outputs = model(pixel_values=pixel_values, labels=labels)

      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      loss_v = loss.item()
      train_loss += loss_v

      current_step += 1
      progress_bar.update(1)
      summary_writer.add_scalar("train_loss", loss_v, global_step=current_step)